In [ ]:
%load_ext autoreload
%autoreload 2
    
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import random

from scipy.optimize import curve_fit

from functionsFIMS import getAnalysisNumbers
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(1148)
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    binWidth=3
    simData.plotAvalancheSize(binWidth)
    simData.plotAvalancheFits(binWidth)

    thresh = 10
    fitPolya = myPolya(24.701, 0.488)
    efficiency = fitPolya.calcEfficiencyErrs(thresh, 0.671, 0.095)
    print(f'{efficiency[0]:.3f} (+{efficiency[1]:.3f}, {efficiency[2]:.3f})')

In [ ]:
if __name__ == '__main__':
    randInt = random.randint(0, 999)
    simData.plotAvalanche2D(randInt, plotName='Random')
    simData.plotAvalancheSignal(randInt)
    simData.plotSignalvsGain()

In [ ]:
TESTDATA = runData(1172)

In [ ]:
def getLargestRadius(xData, yData):
    """
    TODO
    """
    radius2 = xData**2 + yData**2
    maxRadius = math.sqrt(radius2.max())
    minRadius = math.sqrt(radius2.min())

    return maxRadius, minRadius

    

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.', alpha=.15
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', marker='.', alpha=0.15
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    padLength = runData.getRunParameter('Pad Length')
    pitch = runData.getRunParameter('Pitch')
    axLim = pitch/math.sqrt(3)
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)

In [ ]:
def showIonHist(runData):
    """
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    posIons = posIons[posIons['Initial z']!=posIons['Initial z'].iloc[0]]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]

    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad,
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad,
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad,
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    nominalFieldBundle = plt.Circle(
            (0, 0), runData.getRunParameter('Field Bundle Radius'), 
            facecolor='none', edgecolor='c', lw=2
        )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('All Ion Initial Locations')
    ax = fig.add_subplot(111)
    '''
    ax.hist2d(
        posIons['Initial x'], posIons['Initial y'],
        bins=101, cmin=1
    )
    '''
    '''
    ax.hist2d(
        bfIons['Initial x'], bfIons['Initial y'],
        bins=101, cmin=1
    )
    '''
    ax.hist2d(
        capIons['Initial x'], capIons['Initial y'],
        bins=101, cmin=1
    )

    #ax.add_patch(bfCircle)
    #ax.add_patch(capCircle0)
    #ax.add_patch(capCircle1)
    ax.add_patch(nominalFieldBundle)
    
    runData._plotAddCellGeometry(ax, 'xy')

    padLength = runData.getRunParameter('Pad Length')
    pitch = runData.getRunParameter('Pitch')
    
    largeLim = pitch/math.sqrt(3)
    axLim = padLength
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    #ax.legend()

    return fig

_ = showIonHist(TESTDATA)

In [ ]:
def getLargestRadius(xData, yData):
    """
    TODO
    """
    radius2 = xData**2 + yData**2
    maxRadius = math.sqrt(radius2.max())
    minRadius = math.sqrt(radius2.min())

    return maxRadius, minRadius

    

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.'
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', marker='.'
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    #axLim = runData.getRunParameter('Pad Length')
    #ax.set_xlim(-axLim, axLim)
    #ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)

In [ ]:
gridPixData = runData(20008)
FIMSData = runData(1173)

In [ ]:
_ = gridPixData.plotAvalancheFits(binWidth=50)
_ = FIMSData.plotAvalancheFits(binWidth=1)

In [ ]:
efficiencyRuns = np.arange(1200, 1205)

rawEff = []
rawEffErr = []
rawGain = []
fieldRatio = []

for inRun in efficiencyRuns:
    inRunData = runData(inRun)

    rawEff.append(inRunData.getRunParameter('Raw Efficiency (10e)'))
    rawEffErr.append(inRunData.getRunParameter('Raw Efficiency Error'))
    rawGain.append(inRunData.getRunParameter('Raw Gain'))
    fieldRatio.append(inRunData.getRunParameter('Electric Field Ratio'))

rawEff = np.array(rawEff)
rawEffErr = np.array(rawEffErr)
rawGain = np.array(rawGain)
fieldRatio = np.array(fieldRatio)

fig = plt.figure(figsize=(12, 4))
fig.suptitle('FIMS Efficiency')
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)


ax1.errorbar(
    fieldRatio, rawGain, yerr=np.sqrt(rawGain),
    label=f'Raw Gain', ls=''
)

ax2.errorbar(
    fieldRatio, rawEff, yerr=rawEffErr,
    label='Raw Efficiency (10e)', ls=''
)

ax3.errorbar(
    rawGain, rawEff, xerr=np.sqrt(rawGain), yerr=rawEffErr,
    label='Raw Efficiency (10e)', ls=''
)

#ax1.plot([15, 60], [1, 1e3], label='Exponential', c='r', ls=':')
ax2.axhline(y=.95, label='95% Target', c='g', ls=':')
ax3.axhline(y=.95, label='95% Target', c='g', ls=':')


ax1.set_xlabel('Field Ratio')
ax1.set_ylabel('Raw Gain')
ax1.set_yscale('log')

ax2.set_xlabel('Field Ratio')
ax2.set_ylabel('Raw Efficiency (10e)')

ax3.set_xlabel('Raw Gain')
ax3.set_ylabel('Raw Efficiency (10e)')
ax3.set_xscale('log')

ax1.grid()
ax2.grid()
ax3.grid()

plt.tight_layout()
plt.show()



In [ ]:
def mySigmoid(x, k, x0):
    return 1/(1+np.exp(-k*(x-x0)))

p0 = [np.median(rawEff), np.median(fieldRatio)]
popt, pcov = curve_fit(mySigmoid, fieldRatio, rawEff, p0, method='lm', sigma=rawEffErr, absolute_sigma=True)

print(f'Fit parameters: {popt[0]:.3f}, {popt[1]:.3f}')

fitFields = np.arange(min(fieldRatio)-10, max(fieldRatio)+10)
fitSigmoid = mySigmoid(fitFields, popt[0], popt[1])

plt.errorbar(fieldRatio, rawEff, yerr=rawEffErr, ls='', label='Simulated Raw Efficiency (10e)')
plt.plot(fitFields, fitSigmoid, label='Sigmoid Fit', c='r', ls='--')
plt.axhline(y=.95, label='95% Target', c='g', ls=':')

plt.title('FIMS Efficiency Fit')
plt.legend()
plt.grid()
plt.show()